In [2]:
import ee
import geemap
from datetime import datetime

In [3]:
geemap.ee_initialize()
oeel = geemap.requireJS()
ee.Authenticate()
ee.Initialize(project='ee-amazyar-test1')

start_date = ee.Date("1990-03-16") # с 1990 тепловая информация Landsat-5 без колебаний
end_date = ee.Date(datetime.now())
openwater_month_start = 6 # июнь
openwater_month_end = 10 # октябрь

baikal_shape = ee.FeatureCollection('projects/ee-amazyar-test/assets/baikal')

(node:33802) [DEP0040] DeprecationWarning: The `punycode` module is deprecated. Please use a userland alternative instead.
(Use `node --trace-deprecation ...` to show where the warning was created)


!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!
!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!
!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!
!!!!! On 11-07-2022 this code has been updated to run with landsat collection 2 data !!!!!


In [4]:
x_min = 103.59119823586846
y_min = 51.317386807273074
x_max = 109.85340526711846
y_max = 55.789384409348855

baikal_rect = ee.Geometry.Rectangle([
   [x_min, y_min],
   [x_max, y_max]
   ])

In [5]:
LandsatLST = geemap.requireJS("../baikal_vkr_init/Landsat_LST.js")

In [6]:
use_ndvi = True

def get_landsat_collection(satellite_name: str):
   coll = LandsatLST.collection(satellite_name, start_date, end_date, baikal_rect, use_ndvi)#.filter(ee.Filter.lt('CLOUD_COVER', 30))
   return coll.select('LST')

In [7]:
landsat = ee.ImageCollection.merge(get_landsat_collection('L8'),get_landsat_collection('L9'))
landsat = ee.ImageCollection.merge(landsat,get_landsat_collection('L7'))
landsat = ee.ImageCollection.merge(landsat,get_landsat_collection('L5'))

Отображение LST в RGB

In [8]:
projection = landsat.first().select('LST').projection().getInfo()
projection

{'type': 'Projection',
 'crs': 'EPSG:32649',
 'transform': [30, 0, 420285, 0, -30, 6310515]}

Должно определять минимальную и максимальную температуру, но не работает: выдаёт выбросы (-68.1567823259467 74.04202518385841)

In [10]:
min_landsat = landsat.min()
max_landsat = landsat.max()

In [15]:
some_point_in_baikal_center = ee.Geometry.Point([108.29910984265435,53.62839159543701])

min_lst = min_landsat.reduceRegion(
   reducer=ee.Reducer.min(),
   geometry=some_point_in_baikal_center,
   crs=projection['crs'],
   crsTransform=projection['transform']
).getInfo()['LST']

max_lst = max_landsat.reduceRegion(
   reducer=ee.Reducer.max(),
   geometry=some_point_in_baikal_center,
   crs=projection['crs'],
   crsTransform=projection['transform']
).getInfo()['LST']

print(min_lst, max_lst)

-68.1567823259467 74.04202518385841


In [9]:
cmap = ['blue', 'cyan', 'green', 'yellow', 'red'];
min_lst = 0
max_lst = 23 # источник - википедия

def lst_to_rgb(image: ee.Image):
   return image.visualize(
      bands=['LST'],
      min=min_lst,
      max=max_lst,
      palette=cmap,
      forceRgbOutput=True
      )

landsat_rgb = landsat.map(lst_to_rgb)

In [8]:
rgbed_info = landsat_rgb.first().getInfo()
rgbed_info

{'type': 'Image',
 'bands': [{'id': 'vis-red',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [7711, 7641],
   'crs': 'EPSG:32649',
   'crs_transform': [30, 0, 420285, 0, -30, 6310515]},
  {'id': 'vis-green',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [7711, 7641],
   'crs': 'EPSG:32649',
   'crs_transform': [30, 0, 420285, 0, -30, 6310515]},
  {'id': 'vis-blue',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 255},
   'dimensions': [7711, 7641],
   'crs': 'EPSG:32649',
   'crs_transform': [30, 0, 420285, 0, -30, 6310515]}],
 'properties': {'system:annotations': None,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[112.44079191616753, 54.888964176734376],
    [112.49285242867761, 54.8793527519868],
    [112.57107732932197, 55.019394331736045],
    [112.63056590402482, 55.12563928532152],
    [112.941352

In [13]:
task = ee.batch.Export.video.toDrive(
   collection=landsat_rgb,
   folder="VKR",
   fileNamePrefix="TemperatureVideo",
   framesPerSecond=100,
   region=baikal_rect,
   dimensions=720,
   maxFrames=landsat_rgb.size().getInfo(),
   maxPixels=100000000000,
   crs=projection['crs'],
   crsTransform=projection['transform'],
)
task.start()

In [27]:
Map = geemap.Map()

landsat_LST_vis = {
   'min': 0,
   'max': 23,
   'bands': ['LST'],
   'palette': cmap
}

landsat_rgb_LST_vis = {
   'min': 0,
   'max': 23,
   'bands': ['vis-red', 'vis-green', 'vis-blue']
}

In [ ]:
image_n = 10
displayed_image = landsat_rgb.toList(image_n).get(-1)

In [ ]:
Map.addLayer(ee.Image(displayed_image), landsat_rgb_LST_vis, 'first')
Map